In [1]:
from qiskit import *
import copy
import matplotlib as mpl
import math
from operator import itemgetter

#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram

In [2]:
# 4 by 4 board, 0-3 top down, 0-3 left right.
# Classical Setup to intialize the board in preparation of applying Grovers Algorithm.
# Init creates an initial Quantum circuit, finds all the squares the king attacks. 
def Init(WK):
    qc = QuantumCircuit(17, 16)
    test = [0]*16;
    List = [];
    if (WK[0] == 0 and WK[1] == 0):
        attacked = [[0,0],[0,1],[1,0],[1,1]]
    elif (WK[0] == 3 and WK[1] == 0):
        attacked = [[3,0],[3,1],[2,0],[2,1]]
    elif (WK[0] == 3 and WK[1] == 3):
        attacked = [[3,3],[3,2],[2,3],[2,2]]
    elif (WK[0] == 0 and WK[1] == 3):
        attacked = [[0,3],[1,3],[0,2],[1,2]]
    elif (WK[0] == 0):
        attacked = [[WK[0], WK[1]-1],[WK[0],WK[1]],[WK[0],WK[1]+1],
                    [WK[0]+1, WK[1]-1],[WK[0]+1,WK[1]],[WK[0]+1,WK[1]+1]]
    elif (WK[0] == 3):
        attacked = [[WK[0], WK[1]-1],[WK[0],WK[1]],[WK[0],WK[1]+1],
                    [WK[0]-1, WK[1]-1],[WK[0]-1,WK[1]],[WK[0]-1,WK[1]+1]]
    elif (WK[1] == 0):
        attacked = [[WK[0]-1, WK[1]],[WK[0],WK[1]],[WK[0]+1,WK[1]],
                    [WK[0]-1, WK[1]+1],[WK[0],WK[1]+1],[WK[0]+1,WK[1]+1]]
    elif (WK[1] == 3):
        attacked = [[WK[0]-1, WK[1]],[WK[0],WK[1]],[WK[0]+1,WK[1]],
                    [WK[0]-1, WK[1]-1],[WK[0],WK[1]-1],[WK[0]+1,WK[1]-1]]
    else:
        attacked = [[WK[0]+1,WK[1]-1],[WK[0]+1,WK[1]],[WK[0]+1,WK[1]+1],
                      [WK[0],WK[1]-1],[WK[0],WK[1]],[WK[0],WK[1]+1],
                      [WK[0]-1,WK[1]-1],[WK[0]-1,WK[1]],[WK[0]-1,WK[1]+1]]
    for i in range(len(attacked)):
        test[4*attacked[i][0]+attacked[i][1]] = 1;
        qc.x(4*attacked[i][0]+attacked[i][1]);
        qc.h(4*attacked[i][0]+attacked[i][1]);
        List.append(4*attacked[i][0]+attacked[i][1]);
    return test, qc, List;
King = [3, 1]
test1, qc, listtest = Init(King);
# Moveable determines all the squares the queen can potentially move to, 
# as well as all the squares the queen can attack from any given position.
def Moveable(Queen, testing, king):
    QueenMoves = []
    test = testing.copy();
    if (test[4*Queen[0]+Queen[1]] == 0):
            boolean = 1;
    else:
        boolean = 0;
    looper = [0, 1, 2, 3];
    for i in range(len(looper)):
        if (4*king[0]+king[1] == 4*looper[i]+Queen[1]):
            break;
        test[4*looper[i]+Queen[1]] = 1;
        QueenMoves.append([i, Queen[1]])
    for i in range(len(looper)):
        if (4*king[0]+king[1] == 4*Queen[0]+looper[i]):
            break;
        test[4*Queen[0]+looper[i]] = 1;
        QueenMoves.append([Queen[0], i]);
    for i in range(len(looper)):
        if (Queen[0]+i < 4 and Queen[0]+i >=0 and Queen[1]-i < 4 and Queen[1]-i >= 0):
            test[4*(Queen[0]+i)+Queen[1]-i] = 1;
            QueenMoves.append([Queen[0]+i, Queen[1]-i]);
        if (Queen[0]-i < 4 and Queen[0]-i >=0 and Queen[1]+i < 4 and Queen[1]+i >= 0):
            test[4*(Queen[0]-i)+Queen[1]+i] = 1;
            QueenMoves.append([Queen[0]-i, Queen[1]+i]);
        if (Queen[0]+i < 4 and Queen[0]+i >=0 and Queen[1]+i < 4 and Queen[1]+i >= 0):
            test[4*(Queen[0]+i)+Queen[1]+i] = 1;
            QueenMoves.append([Queen[0]+i, Queen[1]+i]);
        if (Queen[0]-i < 4 and Queen[0]-i >=0 and Queen[1]-i < 4 and Queen[1]-i >= 0):
            test[4*(Queen[0]-i)+Queen[1]-i] = 1;
            QueenMoves.append([Queen[0]-i, Queen[1]-i]);
        if (boolean == 1):
            test[4*Queen[0]+Queen[1]] = 0;
    return test, QueenMoves;
QueenSpot = [3, 0];
test3, QueenMoves = Moveable(QueenSpot, test1, King);
test4 = np.reshape(test3, (4, 4));
print(test4)
# Checkmate finds all the squares that need to be attacked for a checkmate condition to be met.
# This checkmate condition is then applied later on as a mct gate.
def Checkmate(BK):
    test = [0]*16;
    counter = 0;
    if (BK[0] == 0 and BK[1] == 0):
        attacked = [[0,0],[0,1],[1,0],[1,1]]
        holder = 0
    elif (BK[0] == 3 and BK[1] == 0):
        attacked = [[3,0],[3,1],[2,0],[2,1]]
        holder = 0
    elif (BK[0] == 3 and BK[1] == 3):
        attacked = [[3,3],[3,2],[2,3],[2,2]]
        holder = 0
    elif (BK[0] == 0 and BK[1] == 3):
        attacked = [[0,3],[1,3],[0,2],[1,2]]
        holder = 0
    elif (BK[0] == 0):
        attacked = [[BK[0], BK[1]-1],[BK[0],BK[1]],[BK[0],BK[1]+1],
                    [BK[0]+1, BK[1]-1],[BK[0]+1,BK[1]],[BK[0]+1,BK[1]+1]]
        holder = 1;
    elif (BK[0] == 3):
        attacked = [[BK[0], BK[1]-1],[BK[0],BK[1]],[BK[0],BK[1]+1],
                    [BK[0]-1, BK[1]-1],[BK[0]-1,BK[1]],[BK[0]-1,BK[1]+1]]
        holder = 1;
    elif (BK[1] == 0):
        attacked = [[BK[0]-1, BK[1]],[BK[0],BK[1]],[BK[0]+1,BK[1]],
                    [BK[0]-1, BK[1]+1],[BK[0],BK[1]+1],[BK[0]+1,BK[1]+1]]
        holder = 1;
    elif (BK[1] == 3):
        attacked = [[BK[0]-1, BK[1]],[BK[0],BK[1]],[BK[0]+1,BK[1]],
                    [BK[0]-1, BK[1]-1],[BK[0],BK[1]-1],[BK[0]+1,BK[1]-1]]
        holder = 1;
    else:
        attacked = [[BK[0]+1,BK[1]-1],[BK[0]+1,BK[1]],[BK[0]+1,BK[1]+1],
                      [BK[0],BK[1]-1],[BK[0],BK[1]],[BK[0],BK[1]+1],
                      [BK[0]-1,BK[1]-1],[BK[0]-1,BK[1]],[BK[0]-1,BK[1]+1]]
        holder = 2;
    if (holder == 0):
        ret = [0]*4
    elif(holder == 1):
        ret = [0]*6
    else:
        ret = [0]*9
    for i in range(len(attacked)):
        test[4*attacked[i][0]+attacked[i][1]] = 1;
    for n in range(len(test)):
        if (test[n] == 1):
            ret[counter]=n;
            counter = counter + 1;
    return ret;
# The following code reduces our runtime, by essentially removing all qubits that are guaranteed
# to  be 1, and reducing the amount of qubits our circuit needs on a case by case basis.
clause_qubits = Checkmate([2, 3]);
initiallist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
for element in initiallist:
    if element in listtest:
        initiallist.remove(element)
for element in clause_qubits:
    if element in listtest:
        clause_qubits.remove(element)
# Here we create a library of all legal queen moves from the given starting position, we later
# cross check this with grovers to obtain our final result
def library(QueenMoves):
    List = [];
    for n in range(len(QueenMoves)):
        Position, trash = Moveable(QueenMoves[n], test1, King);
        List.append(Position);
    return List;
PotentialPositions = library(QueenMoves);
Listing = [];
for n in range(len(PotentialPositions)):
    Hold = ''.join(str(PotentialPositions[n]).split(', '));
    Hold1 = ''.join(str(Hold).split('['));
    Hold2 = ''.join(str(Hold1).split(']'));
    Listing.append(Hold2);
print(Listing)

[[1 0 0 1]
 [1 0 1 0]
 [1 1 1 0]
 [1 1 1 0]]
['0111110011101111', '1100011111101110', '1010110011111110', '1001101011101110', '1001101011101110', '1001101011101110', '1001101011101110', '1001101011101110', '1001101011101110', '0101111011111110', '0111110111111110', '1110001111111111']


In [3]:
#apply a universal hadamard to intialize our circuit, as well as applying the |-> state to ancilla qubit
qc.h([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
qc.x(16)
qc.h(16)

In [11]:
# Here we create the diffuser and oracle, the oracle is an mct on all the squares that need to be attacked
# for checkmate to be achieved. We then make this into a gate so that we can apply as many times as necessary
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s
qc1 = QuantumCircuit(17);
qc1.mct(clause_qubits, 16)
qc1.barrier()
diff = diffuser(len(initiallist))
qc1.append(diff,initiallist)
qc1.decompose().draw(output = 'mpl')
#qc1.to_gate()


ImportError: The class MatplotlibDrawer needs pylatexenc. to install, run "pip install pylatexenc".

In [5]:
count = 0
n = 2**10;
m = 2**6;
# Here we calculate m and n so we know how many times our circuit should be ran for optimal results
while (count <= (math.pi/4)*math.sqrt(n/m)):
    count = count + 1
    qc.append(qc1,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])

In [6]:
# Here we measure the resulting qubit states
qc.measure([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])


In [7]:
# We execute our circuit and find the grover solutions
backend = Aer.get_backend('qasm_simulator')
job=execute(qc, backend, shots = 10000)
result1 = job.result()
counts3 = result1.get_counts()
counts4 = {c:counts3[c] for c in counts3 if counts3[c] > 0}
N = 2**(6)+3;
res = dict(sorted(counts4.items(), key = itemgetter(1), reverse = True)[:N])
plot_histogram(res)
res
groversitems = list(dict.keys(res));
groversitems



['1111111111010100',
 '1111111111000011',
 '1111111111001110',
 '1111111111010011',
 '1111111111001011',
 '1111111111011100',
 '1111111111100011',
 '1111111111000010',
 '1111111111001001',
 '1111111111101111',
 '1111111111110100',
 '1111111111000000',
 '1111111111010101',
 '1111111111011111',
 '1111111111100010',
 '1111111111101011',
 '1111111111101100',
 '1111111111110010',
 '1111111111111101',
 '1111111111000001',
 '1111111111100110',
 '1111111111110000',
 '1111111111110101',
 '1111111111111111',
 '1111111111010010',
 '1111111111010111',
 '1111111111101010',
 '1111111111000111',
 '1111111111100001',
 '1111111111100101',
 '1111111111110001',
 '1111111111111000',
 '1111111111111011',
 '1111111111001101',
 '1111111111011000',
 '1111111111111110',
 '1111111111100111',
 '1111111111101000',
 '1111111111101101',
 '1111111111110110',
 '1111111111000110',
 '1111111111001010',
 '1111111111001100',
 '1111111111010001',
 '1111111111011001',
 '1111111111011011',
 '1111111111011110',
 '11111111111

In [8]:
# Due to how the circuit is outputed, we have to reverse every string from the grovers solutions,
# we then cross check it with our legal queen move library created initially to find our final
# checkmates.
grovers = [];
for element in groversitems:
    stringlength=len(element) # calculate length of the list
    slicedString=element[stringlength::-1] # slicing
    grovers.append(slicedString);
final = list(set(grovers) & set(Listing));

QueenMates = [];
for n in range(len(Listing)):
    if Listing[n] in final:
        QueenMates.append(QueenMoves[n])
print(final)
# We print the square the queen needs to move to in order to deliver checkmate
print(QueenMates)

['1110001111111111']
[[0, 3]]
